# Batch Image Processing: Otsu and Adaptive Gaussian Thresholding

This notebook processes all images in a folder using two thresholding methods:
1. **Otsu Global Thresholding**
2. **Adaptive Gaussian Thresholding**

Processed images are saved to separate output folders with the same filenames.


In [1]:
import cv2
import numpy as np
import os
from pathlib import Path


In [2]:
def compute_otsu_threshold(gray_image: np.ndarray) -> int:
    """Compute Otsu's threshold by maximizing between-class variance."""
    hist, _ = np.histogram(gray_image.flatten(), bins=256, range=(0, 256))
    hist = hist.astype(float)
    total_pixels = gray_image.size
    prob = hist / total_pixels
    
    omega = np.cumsum(prob)
    mu = np.cumsum(prob * np.arange(256))
    mu_total = mu[-1]
    
    valid_indices = (omega > 0) & (omega < 1)
    between_class_variance = np.zeros(256)
    between_class_variance[valid_indices] = (
        (mu_total * omega[valid_indices] - mu[valid_indices]) ** 2 / 
        (omega[valid_indices] * (1 - omega[valid_indices]))
    )
    
    optimal_threshold = np.argmax(between_class_variance)
    return optimal_threshold


def otsu_global_thresholding(image: np.ndarray, 
                             blur_ksize: int = 5,
                             morph_ksize: int = 3) -> np.ndarray:
    """Apply Otsu's global thresholding with preprocessing and morphological operations."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (blur_ksize, blur_ksize), 0)
    threshold = compute_otsu_threshold(blurred)
    _, binary = cv2.threshold(blurred, threshold, 255, cv2.THRESH_BINARY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (morph_ksize, morph_ksize))
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    result = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
    
    return result


def adaptive_gaussian_thresholding(image: np.ndarray,
                                   window_size: int = 25,
                                   C: int = 5,
                                   morph_ksize: int = 3) -> np.ndarray:
    """Apply adaptive Gaussian thresholding with local neighborhood statistics."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(
        gray,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        window_size,
        C
    )
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (morph_ksize, morph_ksize))
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    result = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
    
    return result


In [3]:
# Input folder path (folder containing JPG images)
input_folder = 'images'  # Change this to your input folder path

# Output folder paths
output_folder_otsu = 'dichen_output_otsu'
output_folder_adaptiveGaussian = 'dichen_output_adaptiveGaussian'

# Create output directories if they don't exist
os.makedirs(output_folder_otsu, exist_ok=True)
os.makedirs(output_folder_adaptiveGaussian, exist_ok=True)


In [4]:
# Process all images in the input folder
input_path = Path(input_folder)
image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}

image_files = [f for f in input_path.iterdir() if f.suffix in image_extensions]

print(f"Found {len(image_files)} images to process")

for img_file in image_files:
    # Read image
    image = cv2.imread(str(img_file))
    if image is None:
        print(f"Warning: Could not read {img_file.name}, skipping...")
        continue
    
    # Process with Otsu method
    otsu_result = otsu_global_thresholding(image)
    otsu_output_path = os.path.join(output_folder_otsu, img_file.name)
    cv2.imwrite(otsu_output_path, otsu_result)
    
    # Process with Adaptive Gaussian method
    adaptive_result = adaptive_gaussian_thresholding(image)
    adaptive_output_path = os.path.join(output_folder_adaptiveGaussian, img_file.name)
    cv2.imwrite(adaptive_output_path, adaptive_result)
    
    print(f"Processed: {img_file.name}")

print("\nProcessing complete!")


Found 101 images to process
Processed: IMG_1862.jpg
Processed: IMG_1865.jpg
Processed: IMG_1901.jpg
Processed: IMG_1859.jpg
Processed: IMG_1906.jpg
Processed: IMG_1908.jpg
Processed: IMG_1881.jpg
Processed: IMG_1850.jpg
Processed: IMG_1851 (1).jpg
Processed: IMG_1886.jpg
Processed: IMG_1857.jpg
Processed: IMG_1855 (1).jpg
Processed: IMG_1846.jpg
Processed: IMG_1897.jpg
Processed: IMG_1893 (1).jpg
Processed: IMG_1897 (1).jpg
Processed: IMG_1841.jpg
Processed: IMG_1890.jpg
Processed: IMG_1919.jpg
Processed: IMG_1833.jpg
Processed: IMG_1917.jpg
Processed: IMG_1848.jpg
Processed: IMG_1899.jpg
Processed: IMG_1910.jpg
Processed: IMG_1848 (1).jpg
Processed: IMG_1834.jpg
Processed: IMG_1887.jpg
Processed: IMG_1856.jpg
Processed: IMG_1892 (1).jpg
Processed: IMG_1909.jpg
Processed: IMG_1896 (1).jpg
Processed: IMG_1851.jpg
Processed: IMG_1849 (1).jpg
Processed: IMG_1907.jpg
Processed: IMG_1900.jpg
Processed: IMG_1845 (1).jpg
Processed: IMG_1889.jpg
Processed: IMG_1858.jpg
Processed: IMG_1864.jpg
